## Two extensions to the Kelly Criteria

Today provides a couple of quick and easy results, again motivated by Thorpe's discussion with Buffett, as well as a few other papers.

In their discussion, Thorpe gave an example of investing in stocks where there were three outcomes.  This was somewhat exciting, to think he would provide a solution for a three-outcome setup.  Instead, the middle outcome was set to zero and he used the standard Kelly setup.  Today, we will look at two extensions to Kelly:

- [What if p and q don't add up to one?](#first)
- [Adding a risk free rate](#second)

---
<a id='first'></a>
### What if p and q don't add up to one?

We start the derivation just as we did <a href="https://github.com/ghstmann/Kelly-Criterion/blob/main/Basic%20Kelly.ipynb">last week</a>.  (see last week's post for the Thorpe link) Specifically, we <u>do not</u> use the logarithmic transform, we take the first derivative, and we solve for the $f$ that sets it to zero. (I also substituted out $\rho$ for $p$ to make programming easier.)

In [6]:
import sympy as sp

In [13]:
f, b1, b2, p, q = sp.symbols('f b1 b2 p q')
alt_kelly = (1 + b1 * f)**p * (1 + b2 * f)**q

alt_kelly

(b1*f + 1)**p*(b2*f + 1)**q

In [17]:
# taking the first derivative
first = sp.diff(alt_kelly, f)
solution = sp.simplify(sp.solve(first, f)[0])

# a little light formatting to make it look more familiar
solution = solution * (q + p)
solution = sp.simplify(solution)
a = solution.as_ordered_terms()
a = [x/(q + p) for x in a]
a = [sp.simplify(x) for x in a]
solution = sum(a)
solution

-p/(b2*(p + q)) - q/(b1*(p + q))

This is exactly the same as the Kelly solution except that the probabilities are normalized by their sum.  In other words, both $p$ and $q$ are treated as if they add up to one.  For Thorpe's example, it is as if the middle outcome never even existed.

Let's see an example with the following inputs:

- $b_1 = 2$
- $b_2 = -1$
- $p = 0.3$
- $q = 0.2$

In [24]:
b1, b2, p, q = 2, -1, 0.3, 0.2

f = -p / b2 / (p + q) - q / b1 / (p + q)
print(f'The optimal investment ratio is {f:.2f}.')

The optimal investment ratio is 0.40.


In [29]:
# What happens if we divide both p and q by 100?

b1, b2, p, q = 2, -1, 0.003, 0.002

f = -p / b2 / (p + q) - q / b1 / (p + q)
print(f'The optimal investment ratio is {f:.2f}.')

The optimal investment ratio is 0.40.


This is a very strange result.  If we assume $p$ is 60% and $q$ is 40%.  It makes sense that someone would be willing to bet 40% of their investment purse.  Once $p$ and $q$ are divided by two, that means half the time, the better would win or lose nothing.  It is harder to argue that someone would take that bet.  

Going further, the last example has no winnings or losses 99.5% of the time.  The better must have other decent options to invest their money.  This brings up the idea of an opportunity cost.  The following section will explore the Kelly Criterion when the better earns an interest rate on funds not in the bet.

<a id='second'></a>
### Adding a Risk Free Rate
This derivation follows <a href="https://www.columbia.edu/~ww2040/PortfolioChoice96.pdf">Porffolio Choice and the Bayesian Kelly Criterion</a> by Browne and Whitt.  That said, this was not a focus of their research and they didn't go into the implications of this method.

The setup for this analysis is that, no matter which path you take, whether success ($p$) or failure ($q$), you always earn $(1 - f)*r$.

$expected\ winnings = (1 + (1 - f)r + f b_1)^p \ (1 + (1 - f)r + f b_2)^q$

This can be rewritten as:

$expected\ winnings = (1 + r + f (b_1 - r))^p \ (1 + r + f (b_2 - r))^q$


In [42]:
f, b1, b2, p, q, r = sp.symbols('f b1 b2 p q r')

alt_kelly_r = (1 + r + (b1 - r) * f)**p * (1 + r + (b2 - r) * f)**q

alt_kelly_r

(f*(b1 - r) + r + 1)**p*(f*(b2 - r) + r + 1)**q

We'll start by taking the first derivative and finding the solution.

In [64]:
first_r = sp.diff(alt_kelly_r, f)
first_r = first_r.as_ordered_terms()
first_r = [x / (1 + r + (b1 - r)*f)**(p - 1) for x in first_r]
first_r = [x / (1 + r + (b2 - r)*f)**(q - 1) for x in first_r]
first_r = [sp.simplify(x) for x in first_r]

first_r = sum(first_r)
first_r

p*(b1 - r)*(f*(b2 - r) + r + 1) + q*(b2 - r)*(f*(b1 - r) + r + 1)

In [78]:
# I'm not an expert in Sympy yet so I'll manually rewrite this, for better manipulation

first_r = f * (p + q)*(b1 - r)*(b2 - r) + (1 + r)*p*(b1 - r) + (1 + r)*q*(b2 -r)
first_r

f*(b1 - r)*(b2 - r)*(p + q) + p*(b1 - r)*(r + 1) + q*(b2 - r)*(r + 1)

In [80]:
# we are looking for f = 0 so it doesn't matter if we get rid of a few common terms

# the following line becomes important later
first_r = first_r.subs(p+q, 1)

first_r = first_r.as_ordered_terms()
first_r = [x / (b1 - r) / (b2 - r) for x in first_r]
first_r = sum(first_r)
first_r

f + p*(r + 1)/(b2 - r) + q*(r + 1)/(b1 - r)

In [97]:
# I could have done this more formally but the solution was obvious

solution_r = -(first_r - f)
solution_r

-p*(r + 1)/(b2 - r) - q*(r + 1)/(b1 - r)

This looks like the normal Kelly solution but now the bet payoffs are adjusted and the entire amount is multiplied by $(1 + r)$.  This is a cool looking solution but it is not clear whether it achieves what we hope.  We want the better fraction in the presence of $r$ to be less than that without.

To show that this is the case, I'll take the derivative with respect to $r$ and show that it is negative for all interesting inputs.

In [231]:
diff_by_r = sp.diff(solution_r, r)
diff_by_r = diff_by_r.as_ordered_terms()

# the following lines simply move a negative sign out of the parenthesis for easier reading
a1 = sp.simplify(sum(diff_by_r[:2]))
a1 = sp.factor(sp.numer(a1))/sp.denom(a1)

a2 = sp.simplify(sum(diff_by_r[2:]))
a2 = sp.factor(sp.numer(a2))/sp.denom(a2)

diff_by_r = a1 + a2
diff_by_r


-p*(b2 + 1)/(b2 - r)**2 - q*(b1 + 1)/(b1 - r)**2

- $b_2$ is always negative and never lower than $-1$.  We can't lose more than we have bet.  This ensures the first term is always negative.
- $b_1$ is always positive so the second term is also always negative.

This proves that introducing a positive interest rate (or other opportunity cost) always reduces the bet fraction.  Unfortunately, as we'll see in the next cell, we still have the problem that $p + q \ne 1$ is an issue.

In [242]:
first_r = f * (p + q)*(b1 - r)*(b2 - r) + (1 + r)*p*(b1 - r) + (1 + r)*q*(b2 -r)

#unlike before, I'm commenting out the following substition
#first_r = first_r.subs(p+q, 1)

# instead, I'll divide it from the first derivative to get the solution
first_r = first_r.as_ordered_terms()
first_r = [x / (b1 - r) / (b2 - r) / (p + q) for x in first_r]
first_r = sum(first_r)
solution = - (first_r - f)
solution

-p*(r + 1)/((b2 - r)*(p + q)) - q*(r + 1)/((b1 - r)*(p + q))

Even though we've adjusted the payoffs and multiplied by (1 + r), we still have a non-sensical result.